<a href="https://colab.research.google.com/github/domingues100/Kaggle---Codes/blob/master/SpaceShip_after_lost_everything.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'spaceship-titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F34377%2F3220602%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240517%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240517T015421Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D221df93dd192cca0779d2f68afa12f97f172469df398912d37216c8e46db0ba5e43e98529e9ee1e120701a5ffc36fdb4db0187a77263d12a41a1945cc59365d32d3912919c2bc6baa3ac049c63fe7b1e560d1a1a2f6cde96f885d23496211fdac6a2a9707a7c557c76df1b53dc59c09a44ce370b5ab2953cd462629c95c0f4355c573537e54a163f1ea0d9c7cc3c1cdd0d0d54cd27fb392d9be677861e1a9ed23f6638dc2146fa2a959272f45c509af1bb134305c276e4f43d3958d99468789f28d6a7f8335fd01382bb8d27a7cdba62ef73d2dda9b2946459592cdba5893cc333d571421b956e80a6eb97dae51dc40d9bc9c1929416220f566280b5a6836605'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
train_csv = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")

In [ ]:
train_csv.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
#creating some features based on Cabin column
train_csv[["Deck", "Num", "Side"]] = train_csv["Cabin"].str.split("/", expand = True)
train_csv.drop(columns = ["Cabin", "Name"], inplace = True)

In [ ]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Transported   8693 non-null   bool   
 12  Deck          8494 non-null   object 
 13  Num           8494 non-null   object 
 14  Side          8494 non-null   object 
dtypes: bool(1), float64(6), object(8)
memory usage: 959.4+ KB


* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck -> fill up with mean
* Age, Deck, Num, Side, CryoSleep, HomePlanet, VIP, Destination -> fill with max appear

In [ ]:
#deal with null and nan
features_to_fill_mean = ["RoomService", "ShoppingMall", "Spa", "VRDeck", "FoodCourt"]
features_to_fill_max_appearances = ["Age", "Deck", "Num", "Side", "CryoSleep", "HomePlanet", "VIP", "Destination"]

train_csv[features_to_fill_mean] = train_csv[features_to_fill_mean].fillna(train_csv[features_to_fill_mean].mean())
train_csv[features_to_fill_max_appearances] = train_csv[features_to_fill_max_appearances].fillna(train_csv[features_to_fill_max_appearances].mode().iloc[0])

/tmp/ipykernel_33/3868937850.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_csv[features_to_fill_max_appearances] = train_csv[features_to_fill_max_appearances].fillna(train_csv[features_to_fill_max_appearances].mode().iloc[0])


In [ ]:
#trying to create new features
train_csv["TotalSpent"] = train_csv[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].sum(axis=1)

#Deal with passenger id gggg_pp -> gggg is the group and pp is the number within the group
train_csv[["Group", "Position"]] = train_csv["PassengerId"].str.split("_", expand = True)

#Feature HomePlanet to End point
train_csv["InitEnd"] = train_csv["HomePlanet"] + "_" + train_csv["Destination"]

#Creating the column with number of people in a group

group_counts = train_csv["Group"].value_counts()
group_counts = group_counts.to_dict()
train_csv["QtGroupMembers"] = train_csv["Group"].map(group_counts)

In [ ]:

train_csv = pd.get_dummies(train_csv, columns=['HomePlanet', 'Deck', 'Side', "Destination", "VIP"])

train_csv.head()

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Num,...,Deck_F,Deck_G,Deck_T,Side_P,Side_S,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0001_01,False,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,False,False,False,True,False,False,False,True,True,False
1,0002_01,False,24.0,109.0,9.0,25.0,549.0,44.0,True,0,...,True,False,False,False,True,False,False,True,True,False
2,0003_01,False,58.0,43.0,3576.0,0.0,6715.0,49.0,False,0,...,False,False,False,False,True,False,False,True,False,True
3,0003_02,False,33.0,0.0,1283.0,371.0,3329.0,193.0,False,0,...,False,False,False,False,True,False,False,True,True,False
4,0004_01,False,16.0,303.0,70.0,151.0,565.0,2.0,True,1,...,True,False,False,False,True,False,False,True,True,False


In [ ]:
columns = train_csv.columns
print(columns)

Index(['PassengerId', 'CryoSleep', 'Age', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'Num', 'TotalSpent',
       'Group', 'Position', 'InitEnd', 'QtGroupMembers', 'HomePlanet_Earth',
       'HomePlanet_Europa', 'HomePlanet_Mars', 'Deck_A', 'Deck_B', 'Deck_C',
       'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_T', 'Side_P', 'Side_S',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e', 'VIP_False', 'VIP_True'],
      dtype='object')


In [ ]:
#now its necessary encode categorical data
train_csv.drop(columns = ["PassengerId"])
le = LabelEncoder()

encodings = ["Transported", "Group", "Position", "InitEnd", 'HomePlanet_Earth', 'HomePlanet_Europa',
            'HomePlanet_Mars', 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G',
            'Deck_T', 'Side_P', 'Side_S', 'CryoSleep','Destination_55 Cancri e',
            'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'VIP_False', 'VIP_True']

for encode in encodings:
    le.fit(train_csv[encode])
    train_csv[encode] = le.transform(train_csv[encode])

In [ ]:
train_csv.head()

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Num,...,Deck_F,Deck_G,Deck_T,Side_P,Side_S,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0001_01,0,39.0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,1,1,0
1,0002_01,0,24.0,109.0,9.0,25.0,549.0,44.0,1,0,...,1,0,0,0,1,0,0,1,1,0
2,0003_01,0,58.0,43.0,3576.0,0.0,6715.0,49.0,0,0,...,0,0,0,0,1,0,0,1,0,1
3,0003_02,0,33.0,0.0,1283.0,371.0,3329.0,193.0,0,0,...,0,0,0,0,1,0,0,1,1,0
4,0004_01,0,16.0,303.0,70.0,151.0,565.0,2.0,1,1,...,1,0,0,0,1,0,0,1,1,0


In [ ]:
corr = abs(train_csv.corr()["Transported"]).sort_values(ascending=False)
print(corr)

Transported                  1.000000
CryoSleep                    0.460132
RoomService                  0.242048
Spa                          0.218791
VRDeck                       0.204825
TotalSpent                   0.199445
HomePlanet_Europa            0.176916
HomePlanet_Earth             0.168845
Deck_B                       0.144733
Destination_55 Cancri e      0.108722
Deck_C                       0.108193
Side_S                       0.101397
Side_P                       0.101397
Deck_E                       0.097965
Destination_TRAPPIST-1e      0.096319
Deck_F                       0.086360
QtGroupMembers               0.082644
Age                          0.074153
InitEnd                      0.073520
Position                     0.066390
FoodCourt                    0.046074
Num                          0.044012
VIP_True                     0.037261
VIP_False                    0.037261
Deck_D                       0.034046
Group                        0.021825
PassengerId 

In [ ]:
# features_selected = ["CryoSleep","RoomService", "Spa", "VRDeck", "TotalSpent", "HomePlanet", "Deck", "Destination", "Side", "QtGroupMembers", "Age", "InitEnd"]
#features_selected = ["CryoSleep", "RoomService", "Spa", "VRDeck", "TotalSpent", "HomePlanet_Europa", "HomePlanet_Earth", "Deck_B", "Deck_C", "Destination", "Side_S", "Side_P"]
features_selected = ["CryoSleep", "RoomService", "Spa", "VRDeck", "TotalSpent", "HomePlanet_Europa", "HomePlanet_Earth", "Deck_B", "Destination_55 Cancri e",
                    "Deck_C", "Side_S"]

Y = train_csv["Transported"]
train_csv = train_csv[features_selected]
train_csv["TotalSpent"] = train_csv["TotalSpent"]/train_csv["TotalSpent"].max()
train_csv["VRDeck"] = train_csv["VRDeck"]/train_csv["VRDeck"].max()
train_csv["Spa"] = train_csv["Spa"]/train_csv["Spa"].max()
train_csv["RoomService"] = train_csv["RoomService"]/train_csv["RoomService"].max()

In [ ]:
# train_csv["Age"] = train_csv["Age"]/train_csv["Age"].max()
# train_csv.head()

In [ ]:
# X_train , X_test , y_train , y_test = train_test_split(train_csv, Y, test_size=0.2)

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, models

# kFold=StratifiedKFold(n_splits = 10, shuffle = True)

# fold = 0
# for train_index,test_index in kFold.split(np.zeros(Y.size),Y):

#     X_train, X_test, y_train, y_test = train_csv.iloc[train_index], train_csv.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]

#     train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
#     train_dataset = train_dataset.shuffle(buffer_size=len(train_dataset)).batch(64)

#     model = models.Sequential([
#             layers.Dense(256, activation='relu', input_shape=(train_csv.shape[1],)),
#             layers.Dropout(rate = 0.5),
#             layers.Dense(128, activation='relu'),
#             layers.Dropout(rate = 0.5),
#             layers.Dense(64, activation='relu'),
#             layers.Dense(1, activation='sigmoid')
#             ])

#     model.compile(optimizer = "Adam",
#                   loss = "binary_crossentropy",
#                   metrics = ["accuracy"])

#     history = model.fit(train_dataset, epochs = 100)

#     model.save(f"model_{fold}.h5")
#     fold+=1

In [ ]:
# from tensorflow.keras.models import load_model

# val = []
# for i in range(10):
#     model = load_model(f"model_{i}.h5")
#     val.append(model.evaluate(X_test, y_test)[1])

# print(sum(val)/len(val))

In [ ]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(learning_rate=0.05, max_depth=10, n_estimators=3000)
#xgboost_model = XGBClassifier(learning_rate=0.05, max_depth=15, n_estimators=3000)
catboost_model = CatBoostClassifier(learning_rate=0.05, max_depth=15, n_estimators=3000, verbose=0)

voting_model = VotingClassifier([('xgb', lgbm_model), ('cb', catboost_model)])

voting_model.fit(train_csv, Y)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=2000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('cb',
                              <catboost.core.CatBoostClassifier object at 0x7ce876804df0>)])

# Test prediction

In [ ]:
test_csv = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test_csv[["Deck", "Num", "Side"]] = test_csv["Cabin"].str.split("/", expand = True)
test_csv.drop(columns = ["Cabin", "Name"], inplace = True)

In [ ]:
#deal with null and nan
features_to_fill_mean = ["RoomService", "ShoppingMall", "Spa", "VRDeck", "FoodCourt"]
features_to_fill_max_appearances = ["Age", "Deck", "Num", "Side", "CryoSleep", "HomePlanet", "VIP", "Destination"]

test_csv[features_to_fill_mean] = test_csv[features_to_fill_mean].fillna(test_csv[features_to_fill_mean].mean())
test_csv[features_to_fill_max_appearances] = test_csv[features_to_fill_max_appearances].fillna(test_csv[features_to_fill_max_appearances].mode().iloc[0])

/tmp/ipykernel_33/460307614.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_csv[features_to_fill_max_appearances] = test_csv[features_to_fill_max_appearances].fillna(test_csv[features_to_fill_max_appearances].mode().iloc[0])


In [ ]:
#trying to create new features
test_csv["TotalSpent"] = test_csv[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].sum(axis=1)

#Deal with passenger id gggg_pp -> gggg is the group and pp is the number within the group
test_csv[["Group", "Position"]] = test_csv["PassengerId"].str.split("_", expand = True)

#Feature HomePlanet to End point
test_csv["InitEnd"] = test_csv["HomePlanet"] + "_" + test_csv["Destination"]


#Creating the column with number of people in a group

group_counts = test_csv["Group"].value_counts()
group_counts = group_counts.to_dict()
test_csv["QtGroupMembers"] = test_csv["Group"].map(group_counts)

In [ ]:
test_csv = pd.get_dummies(test_csv, columns=['HomePlanet', 'Deck', 'Side', "Destination", "VIP"])

test_csv.head()

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Num,TotalSpent,...,Deck_F,Deck_G,Deck_T,Side_P,Side_S,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0013_01,True,27.0,0.0,0.0,0.0,0.0,0.0,3,0.0,...,False,True,False,False,True,False,False,True,True,False
1,0018_01,False,19.0,0.0,9.0,0.0,2823.0,0.0,4,2832.0,...,True,False,False,False,True,False,False,True,True,False
2,0019_01,True,31.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,False,False,False,False,True,True,False,False,True,False
3,0021_01,False,38.0,0.0,6652.0,0.0,181.0,585.0,1,7418.0,...,False,False,False,False,True,False,False,True,True,False
4,0023_01,False,20.0,10.0,0.0,635.0,0.0,0.0,5,645.0,...,True,False,False,False,True,False,False,True,True,False


In [ ]:
#now its necessary encode categorical data
Passengers = test_csv["PassengerId"]
test_csv.drop(columns = ["PassengerId"])
le = LabelEncoder()

# encodings = ["HomePlanet", "CryoSleep", "Destination", "VIP", "Deck", "Side", "Group", "Position"]
encodings = ["Group", "Position", "InitEnd", 'HomePlanet_Earth', 'HomePlanet_Europa',
            'HomePlanet_Mars', 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G',
            'Deck_T', 'Side_P', 'Side_S', 'CryoSleep','Destination_55 Cancri e',
            'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'VIP_False', 'VIP_True']

for encode in encodings:
    le.fit(test_csv[encode])
    test_csv[encode] = le.transform(test_csv[encode])

In [ ]:
features_selected = ["CryoSleep", "RoomService", "Spa", "VRDeck", "TotalSpent", "HomePlanet_Europa", "HomePlanet_Earth", "Deck_B", "Destination_55 Cancri e",
                    "Deck_C", "Side_S"]

test_csv = test_csv[features_selected]
test_csv["TotalSpent"] = test_csv["TotalSpent"]/test_csv["TotalSpent"].max()
test_csv["VRDeck"] = test_csv["VRDeck"]/test_csv["VRDeck"].max()
test_csv["Spa"] = test_csv["Spa"]/test_csv["Spa"].max()
test_csv["RoomService"] = test_csv["RoomService"]/test_csv["RoomService"].max()
# test_csv["Age"] = test_csv["Age"]/test_csv["Age"].max()

In [ ]:
test_csv.head()

,CryoSleep,RoomService,Spa,VRDeck,TotalSpent,HomePlanet_Europa,HomePlanet_Earth,Deck_B,Destination_55 Cancri e,Deck_C,Side_S
0,1,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,1
1,0,0.000000,0.142260,0.000000,0.084120,0,1,0,0,0,1
2,1,0.000000,0.000000,0.000000,0.000000,1,0,0,1,1,1
3,0,0.000000,0.009121,0.026266,0.220341,1,0,0,0,1,1
4,0,0.000865,0.000000,0.000000,0.019159,0,1,0,0,0,1


In [ ]:
# from tensorflow.keras.models import load_model

# preds = []
# for i in range(10):
#     model = load_model(f"model_{i}.h5")
#     predictions = model.predict(test_csv)
#     #predictions = np.where(predictions > 0.5, True, False)
#     preds.append(predictions)


In [ ]:
# preds_mean = np.mean(preds, axis=0)
# preds_mean = np.where(preds_mean > 0.5, True, False)

In [ ]:
# preds_mean = preds_mean.tolist()
# preds_mean = [item[0] for item in preds_mean]

In [ ]:
test_predictions = voting_model.predict(test_csv)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [ ]:
print(test_predictions)

[1 0 1 ... 1 1 1]


In [ ]:
subs = {"PassengerId": Passengers, "Transported": test_predictions}
subs = pd.DataFrame(subs)
subs = subs.replace({1: True, 0: False})

subs.to_csv("submission.csv", index=False)

/tmp/ipykernel_33/3986478319.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  subs = subs.replace({1: True, 0: False})


In [ ]:
subs.head()